In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# ChatOpenAI 모델을 초기화합니다.
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 시를 설명하는 체인 1: 프로그래밍 언어에 대한 시를 쓰는 체인
programming_poetry_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a poet specialized in writing poems about programming languages. Write a poem about {programming_language}.",
        ),
        ("human", ""),
    ]
)

programming_poetry_chain = programming_poetry_prompt | chat

# 시를 설명하는 체인 2: 프로그래밍 언어를 설명하는 체인
programming_language_description_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are explaining about the {programming_language} programming language. Describe its features and use cases.",
        ),
        ("human", ""),
    ]
)

programming_language_description_chain = programming_language_description_prompt | chat

# 두 개의 체인을 연결합니다.
final_chain = {
    "programming_language": programming_poetry_chain
} | programming_language_description_chain

# 최종 체인을 호출하고 프로그래밍 언어의 이름을 받아 시와 설명을 출력합니다.
final_chain.invoke({"programming_language": "Python"})